In [1]:
%display latex

In [33]:
N = 3
F = GF(2^3, 'x')
x = F.gen()

basis = [x^3, x^5, x^6] # self-dual basis

# express an element as a linear combination of the basis
components = lambda k: [(k * el).trace() for el in basis]

# additive group character
chi = lambda k: exp(pi * I * int(k.trace()))

def toInt(k):
    return list(F).index(k)

y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

for mu in F:
    curve = lambda t: mu * t # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [1,1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


In [34]:
def powers(pts):
    return [(toInt(p[0]), toInt(p[1])) for p in pts]

In [35]:
def ray(mu):
    return lambda t: mu * t

def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [36]:
ps = PS(ray, [[1,1,1]] * 8)
ps

[ 1  1  1  1  1  1  1  1]
[ 1 -1 -I  1 -I  I  I -1]
[ 1 -I -1  I -I  I  1 -1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[ 1  I  I  1  1  I  1  I]
[ 1  I  1  1  I  1  I  I]
[ 1 -1 -1  I -1  I  I -I]

In [37]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

In [38]:
def testCurve(points):
    for p1 in points:
        for p2 in points:
            p_op = D(p1[0], p1[1]) * D(p2[0], p2[1])
            s_op = D(p1[0] + p2[0], p1[1] + p2[1]) # assuming additivity
            if p_op != s_op:
                raise Exception(p1, p2)
    return True

In [39]:
def checkAdditivity(points):
    for p1 in points:
        for p2 in points:
            p3 = (p1[0] + p2[0], p1[1] + p2[1])
            if p3 not in points:
                raise Exception
    return True

def checkIsotropy(points):
    for p1 in points:
        for p2 in points:
            if (p1[0]*p2[1] - p1[1]*p2[0]).trace() != 0:
                raise Exception
    return True

We have to work backwards to obtain the curve. The first three sets of operators Andres sent us are the straight lines $\alpha = 0$, $\beta = 0$ and $\beta = \alpha$, which are trivially abelian. The fourth curve is given by the set of points:
$$
\{(0,0), (x^5,x^6), (x^3+x^5,x^3), (x^3,x^3+x^6), (x^3+x^5+x^6,x^5+x^6), (x^5+x^6,x^3+x^5), 
(x^3+x^6,x^5), (x^6,x^3+x^5+x^6).
\}
$$

In [8]:
points = [(0,0), (x^5,x^6), (x^3+x^5,x^3), (x^3,x^3+x^6), (x^3+x^5+x^6,x^5+x^6), (x^5+x^6,x^3+x^5), 
(x^3+x^6,x^5), (x^6,x^3+x^5+x^6)]

In [9]:
points

[(0, 0),
 (x^2 + x + 1, x^2 + 1),
 (x^2, x + 1),
 (x + 1, x^2 + x),
 (1, x),
 (x, x^2),
 (x^2 + x, x^2 + x + 1),
 (x^2 + 1, 1)]

In [11]:
powers(points)

[(0, 0), (5, 6), (2, 3), (3, 4), (7, 1), (1, 2), (4, 5), (6, 7)]

In [13]:
testCurve(points)

True

In [14]:
def graph(points):
    m = zero_matrix(SR, 2^N)
    for p in points:
        m[toInt(p[0]), toInt(p[1])] = 1
    return m

In [15]:
graph(points)

[1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0]

As we can see this curve is also a straight line $\beta = x \alpha$. The fifth curve is given by:
$$
\{(0,0), (x^6,x^3), (x^5,x^3+x^5), (x^5+x^6,x^5), (x^3,x^5+x^6), (x^3+x^5,x^3+x^6), (x^3+x^6,1), (1,x^6)\}$$

In [42]:
points = [(F(0),F(0)), (x^6,x^3), (x^5,x^3+x^5), (x^5+x^6,x^5), (x^3,x^5+x^6), (x^3+x^5,x^3+x^6), (x^3+x^6,F(1)), (F(1),x^6)]

In [43]:
points

[(0, 0),
 (x^2 + 1, x + 1),
 (x^2 + x + 1, x^2),
 (x, x^2 + x + 1),
 (x + 1, x),
 (x^2, x^2 + x),
 (x^2 + x, 1),
 (1, x^2 + 1)]

In [44]:
powers(points)

[(0, 0), (6, 3), (5, 2), (1, 5), (3, 1), (2, 4), (4, 7), (7, 6)]

In [45]:
graph(points)

[1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0]

In [22]:
testCurve(points)

True

This last curve doesn't seem to be a straight line, but it is still abelian.

In [27]:
for phi0 in F:
    for phi in F:
        pts = [(a, phi0 * a + phi^2 * a^2 + phi * a^4) for a in F]
        if set(pts) == set(points):
            print(phi0, '\t', phi)
            break

x + 1 	 x


In [41]:
graph([(k, x^3 * k + x^2 * k^2 + x * k^4) for k in F])

[1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 1 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0]

In [40]:
testCurve([(k, x^3 * k + x^2 * k^2 + x * k^4) for k in F])

True

In [46]:
x.trace()

0

In [47]:
paulis = ['1', 'Z', 'X', 'Y']

In [52]:
def decomposePoint(p):
    acomps = components(p[0])
    bcomps = components(p[1])
    s = ''
    for i in range(N):
        if acomps[i] == 0 and bcomps[i] == 0:
            s += '1'
        elif acomps[i] == 1 and bcomps[i] == 0:
            s += 'Z'
        elif acomps[i] == 0 and bcomps[i] == 1:
            s += 'X'
        else:
            s += 'Y'
    return s

def decomposePoints(points):
    return [decomposePoint(p) for p in points]

In [53]:
decomposePoints([(k, x^3 * k + x^2 * k^2 + x * k^4) for k in F])

['111', '1YZ', 'YZX', 'ZXX', 'YXY', 'XY1', 'X1Z', 'ZZY']